### Imports

In [2]:
# For Docker image tensorflow/tensorflow:2.14.0-gpu-juptyer and latest-gpu-jupyter
# About the latest tag: sha256:efc25f8ad0ec337e8f4e2de9e7e8e391e6729481c7a7cae4bdea3137da7822c6
!pip install -q emoji
!pip install -q nltk
!pip install -q scikit-learn
!pip install -q transformers
!pip install -q tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 2.3 MB/s eta 0:00:00


In [3]:
from typing import Union
from emoji import demojize, is_emoji
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
import numpy as np
from numpy import ndarray
from sklearn.metrics import f1_score, recall_score
import tensorflow as tf
from transformers import (
    BertConfig,
    BertTokenizer,
    TFBertForSequenceClassification,
)

In [4]:
# The device_name will be used in model.fit()
gpu_devices = tf.config.experimental.list_physical_devices("GPU")

if gpu_devices:
    print("GPU available, using GPU")
    tf.config.experimental.set_visible_devices(gpu_devices[0], "GPU")
    device_name = "/GPU:0"
else:
    print("GPU not available, using CPU")
    device_name = "/CPU:0"

GPU not available, using CPU


### Function for loading data set

In [5]:
def load_text_file(file_path: str) -> list[str]:
    """
    Load a text file and return an array of lines from the file.

    Args:
        file_path: str: The path to the file to load.

    Returns:
        list[str]: An array of lines from the file.
    """
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

### Load the text files

In [6]:
import requests

def load_text_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text.splitlines()
    else:
        print("Failed to fetch data from URL:", url)
        return []

train_text_path: str = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_text.txt"
train_label_path: str = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_labels.txt"

test_text_path: str = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_text.txt"
test_label_path: str = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_labels.txt"

validation_text_path: str = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/val_text.txt"
validation_label_path: str = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/val_labels.txt"

train_text: list[str] = load_text_from_url(train_text_path)
train_label: list[str] = load_text_from_url(train_label_path)

test_text: list[str] = load_text_from_url(test_text_path)
test_label: list[str] = load_text_from_url(test_label_path)

validation_text: list[str] = load_text_from_url(validation_text_path)
validation_label: list[str] = load_text_from_url(validation_label_path)


In [7]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

### Preprocessing

Before implementing transformers, the major way to do data preprocessing was to use stopwords and `WordNetLemmatizer()` from nltk, with a manually created emoji dictionary, which maps emojis to a respective sentiment score.

Tokens are vectorised based on word frequency, and features like sentent length, score from `nltk.sentiment.SentimentIntensityAnalyzer` are also added.

The core function for data processing before implementing transformers:
```python
def text_processing(text: list[str], data: str) -> list[list[str]]:
    """
    Process text data:
    - Remove '@user'
    - Remove hashtags
    - Remove '-'
    - Remove URLs
    - Replace emojis with their corresponding sentiment.
    - Replace punctuation marks.
    - Tokenize the text.
    - Normalize the text with nltk.
    - Lowercase the text.
    # Length of the line is added after vectorization.

    Args:
        text: list[str]: A list of text data.
        data: str: The type of data (train, test, validation).

    Returns:
        list[list[str]]: A list of list of processed text data.
    """
    processed_text = []
    for line in text:
        # Remove '@user'
        line = line.replace("@user", " ")
        # Remove hashtags
        line = line.replace("#", " ")
        # Remove '-'
        line = line.replace("-", " ")
        # Remove URLs (http, https, www)
        line = " ".join([word for word in line.split() if "http" not in word])
        line = " ".join([word for word in line.split() if "www" not in word])
        # Replace emojis with their corresponding sentiment
        line = replace_emojis(line)
        # Replace punctuation marks
        line = punctuation_replacement(line)
        # Tokenize the text.
        tokens = nltk.word_tokenize(line)
        # Normalize the text using WordNetLemmatizer and tokenize the text
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        # Lowercase the text
        tokens = [str(word).lower() for word in tokens]
        processed_text.append(tokens)

    return processed_text
```

More details can be found in commit `63c2c23`, 'removed slang sd', and more variant of this function can be found in the before `63c2c23`.

#### Declere global variables

In [8]:
tokenizer = TweetTokenizer()

### Functions for preprocessing

In [9]:
def text_processing(text: list[str]) -> list[str]:
    """
    Process text data:
    - Remove "@user"
    - Remove "#".
    - Replace "’" and triple dots in one character (…).
    - Tokenize and lowercase.
    - Normalize the tokens and join the line.
    - Replace specific strings.
    - Remove excess space after processing.

    Args:
        text: list[str]: A list of text data.

    Returns:
        list[str]: A list of processed sentences
    """
    processed_text = []
    for line in text:
        # Remove @user
        line = line.replace("@user", "")
        # Remove #
        line = line.replace("#", "")
        # Replace "’" and triple dots in one character (…)
        line = line.replace("’", "'").replace("…", "...")
        # Tokenize and lowercase
        tokens = tokenizer.tokenize(line.lower())
        # Normalize the tokens and join the line
        line = " ".join([normalize_token(token) for token in tokens])
        # Replace specific strings
        line = (
            line.replace("cannot", "can not")
            .replace("can't", "can not")
            .replace("n't ", " not ")
            # Handle cases in English, where when "n't" is replace with
            # " not ", the meaning of the word will be invalid
            .replace("wo not ", "will not ")
            .replace("sha not", "shall not")
            # "ain't" can be "am/is/are not", so it stays
            .replace("ai not ", "ain't ")
        )
        line = (
            line.replace("'m ", " am ")
            .replace("'re ", " are ")
            .replace("'ll ", " will ")
            .replace("'ve ", " have ")
            # 's can mean ownership or "is"
            .replace("'s ", " 's ")
            # 'd can mean "would" or "had"
            .replace("'d ", " 'd ")
        )
        # Remove excess spaces
        line = " ".join(line.split())
        processed_text.append(line)

    return processed_text


def normalize_token(token: str) -> str:
    """
    Normalize the provided token.
    - Replace URLs with "HTTPURL".
    - Replace emojis with respective string.

    Args:
        token: str: The string being normalized

    Return:
        str: The normalized string
    """
    if token.startswith("http") or token.startswith("www"):
        return "HTTPURL"
    elif is_emoji(token):
        return demojize(token)
    else:
        return token

### Process the text data

#### Process all text data

See function docstring from text_processing()

In [10]:
train_text: list[str] = text_processing(train_text)
test_text: list[str] = text_processing(test_text)
validation_text: list[str] = text_processing(validation_text)

##### Convert the labels to integers

In [11]:
validation_labels = [int(x) for x in validation_label]
train_labels = [int(x) for x in train_label]
test_labels = [int(x) for x in test_label]

### Implementation - Transformers

Before implementing transformers, I build a custom netural network for the classification of sentiment labels. Here is the core part of the neural netowk before I implemented transformers:

```python
# Create an Input layer
input_layer = tf.keras.layers.Input(shape=(train_vectorized.shape[1],))

dropout_rate = 0.7
activation_function = "sigmoid"

#tf.random.set_seed(2024)

# Define the model
neural_network = tf.keras.models.Sequential(
    [
        input_layer,
        tf.keras.layers.Dense(2048, activation=activation_function),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(512, activation=activation_function),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(128, activation=activation_function),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(3, activation="softmax"),
    ]
)

# Define early stopping criteria
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, mode="min", verbose=1
)

# Add parameters to Adam optimizer if needed
modified_adam = tf.keras.optimizers.Adam()

neural_network.compile(
    optimizer=modified_adam, loss="categorical_crossentropy", metrics=["accuracy"]
)

rate_scheduler = tf.keras.callbacks.LearningRateScheduler(learning_rate_scheduler)

with tf.device(device_name):
    history = neural_network.fit(
        train_vectorized,
        train_label_one_hot,
        validation_data=(validation_vectorized, validation_label_one_hot),
        epochs=200,
        batch_size=512,
        callbacks=[early_stopping, rate_scheduler],
    )
```

After parameters tunning and testings, the above model can reach a 64% accuracy. More details can be found in commit `63c2c23`, 'removed slang sd', and more variant of this function, parameters and their respective result can be found in the before `63c2c23`.

 ##### Configurable parameters

In [12]:
number_of_epochs: int = 2
batch_size: int = 16
dropout_rate: float = 0.3
learning_rate: float = 0.00001

custom_config = BertConfig.from_pretrained("bert-base-uncased")
custom_config.hidden_dropout_prob = dropout_rate
custom_config.num_labels = 3

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

##### Load pre-trained BERT model and tokenizer

In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-uncased", config=custom_config
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##### Tokenize and pad sequences

In [14]:
train_encodings = tokenizer(
    train_text, padding=True, truncation=True, return_tensors="tf"
)
validation_encodings = tokenizer(
    validation_text, padding=True, truncation=True, return_tensors="tf"
)

##### Convert labels to TensorFlow tensors

In [15]:
train_labels_tensor = tf.convert_to_tensor(tf.one_hot(train_labels, 3))
validation_labels_tensor = tf.convert_to_tensor(tf.one_hot(validation_labels, 3))

##### Define the model

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

##### Variables for logging the history

In [17]:
train_loss: list[float] = []
train_accuracy: list[float] = []
validation_loss: list[float] = []
validation_accuracy: list[float] = []

##### Train the model

Because model.fit() will crash if I put validation_data inside

I created a manual version of history = model.fit(), with validation data

In [ ]:
with tf.device(device_name):
    for i in range(number_of_epochs):
        model.fit(train_encodings, train_labels_tensor, batch_size=batch_size)
        # Get the loss and accuracy after the current epoch
        loss, accuracy = model.evaluate(train_encodings, train_labels_tensor, verbose=0)
        train_loss.append(loss)
        train_accuracy.append(accuracy)
        loss, accuracy = model.evaluate(
            validation_encodings, validation_labels_tensor, verbose=0
        )
        validation_loss.append(loss)
        validation_accuracy.append(accuracy)
        print(f"Epoch {i+1} done. Validation accuracy: {accuracy}, loss: {loss}")

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1230/2851 [===========>..................] - ETA: 7:53:14 - loss: 0.7812 - accuracy: 0.6336

##### Plot the graphs

In [ ]:
# Loss
epochs = range(1, number_of_epochs + 1)
plt.plot(epochs, train_loss, label="Training Loss")
plt.plot(epochs, validation_loss, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Accuracy
plt.plot(epochs, train_accuracy, label="Training Accuracy")
plt.plot(epochs, validation_accuracy, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

### Function for optimising

In [ ]:
def modify_decision_boundary(prediction: ndarray[float], boundary: float) -> list[int]:
    """
    Adjust decision method, make the model tend to predict
    "Netural" more, according to the distribution of the dataset

    If either the score of positive or negative class is in the
    +-boundary range of the netural score, and the remaining class is
    not significant higher that the netural class
    It will be classified as netural

    Args:
        prediction: ndarray[float]: The prediction with socres for each entry.
        boundary: float: The boundary, should be between 0 and 1.

    Return:
        ndarray[int]: The new prediction
    """
    final_result: list[int] = []
    for row in prediction:
        # If the class with the highest score is netural
        if np.max(row) == row[1]:
            final_result.append(1)
            continue

        negative: float = row[0]
        netural: float = row[1]
        positive: float = row[2]

        # Get the netural score range
        min_netural: float = netural * (1 - boundary)
        max_netural: float = netural * (1 + boundary)

        # If score for negative is in netural range and it is the class with highest score
        if min_netural <= negative <= max_netural and np.max(row) == negative:
            final_result.append(1)
        # If score for positive is in netural range and it is the class with highest score
        elif min_netural <= positive <= max_netural and np.max(row) == positive:
            final_result.append(1)
        elif np.max(row) == negative:
            final_result.append(0)
        elif np.max(row) == positive:
            final_result.append(2)

    return final_result

##### Find the best boundary by validation data

In [ ]:
validation_prediction = model.predict(validation_encodings)
prediction_matrix: ndarray[float] = validation_prediction.logits

start = 0.0
boundary: list[float] = []
while start < 1.0:
    final_result = modify_decision_boundary(prediction_matrix, start)
    boundary.append(recall_score(validation_labels, final_result, average="macro"))
    start += 0.01

x_axis = list(range(100))
plt.plot(x_axis, boundary)
plt.xlabel("Boundary percentage")
plt.ylabel("Marco Recall")
plt.title("Recall on validation data with different boundaries")
plt.grid(True)
plt.show()

### Final test on the trained model

##### F1 scores

In [ ]:
test_encodings = tokenizer(
    test_text, padding=True, truncation=True, return_tensors="tf"
)
test_labels = tf.convert_to_tensor(tf.one_hot(test_labels, 3))

netural_range: float = boundary.index(max(boundary)) * 0.01
print(f"Boundary used: {netural_range}")

test_prediction = model.predict(test_encodings)
test_labels = np.argmax(test_labels, axis=1)
prediction_matrix: ndarray[float] = test_prediction.logits

final_result = modify_decision_boundary(prediction_matrix, netural_range)
f1_scores_processed = f1_score(test_labels, final_result, average=None)
macro_average_f1_processed = np.mean(f1_scores_processed)

print("F1 scores for each class:", f1_scores_processed)
print("Macro-averaged F1 score:", macro_average_f1_processed)

##### Recall

I used Marco-averaged recall as the metric to evaluate our model, according to [this paper](https://arxiv.org/pdf/2010.12421).

In [ ]:
recall_per_class = recall_score(test_labels, final_result, average=None)
macro_average_recall = sum(recall_per_class) / len(recall_per_class)

print("Recall for each class:", recall_per_class)
print("Macro-averaged recall:", macro_average_recall)

In [ ]:
# model.save('71_recall_test.keras')